In [1]:
import time
import pandas as pd
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup


from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings('ignore')

#to display all rows columns 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

### Step1: Download ChromeDriver:
(check chrome version)
https://chromedriver.chromium.org/downloads

### Step2: Change the chrome drive path below to your local folder

In [2]:
def perform_scraping(innings):
    ##### Web scrapper for infinite scrolling page #####
    driver_path = r"C:\Users\david\Downloads\chromedriver_win32\chromedriver.exe"
    driver = webdriver.Chrome(executable_path=driver_path)

    # Scrape Ball by ball data from any cricinfo commentary
#     match_url = "https://www.espncricinfo.com/series/ipl-2021-1249214/chennai-super-kings-vs-kolkata-knight-riders-final-1254117/ball-by-ball-commentary"
#     match_url = "https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2021-22-1267897/australia-vs-south-africa-13th-match-group-1-1273724/ball-by-ball-commentary"
#     match_url = "https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2021-22-1267897/england-vs-west-indies-14th-match-group-1-1273725/ball-by-ball-commentary"
#     match_url = "https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2021-22-1267897/south-africa-vs-west-indies-18th-match-group-1-1273729/ball-by-ball-commentary"
    match_url = "https://www.espncricinfo.com/series/syed-mushtaq-ali-trophy-2021-22-1280192/vidarbha-vs-karnataka-2nd-semi-final-1280301/ball-by-ball-commentary"
    driver.get(match_url)

    # click drop down menu
    dd = driver.find_element_by_css_selector(".dropdown-container")
    dd.click()

    time.sleep(2)
    # selct item from a drop down menu
    xpath_string = f"//div[@class='ci-dd__menu']/ul[1]/li[{innings}]"
    driver.find_element(By.XPATH, xpath_string).click()


    time.sleep(2)  # Allow 2 seconds for the web page to open
    scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 1

    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break 

    # soup = BeautifulSoup(driver.page_source, "html.parser")
    soup = BeautifulSoup(driver.page_source, "lxml")

    # To get to array of Match Comments - 

    # section - id:pane-main
    # section - id:main-container
    # div - class:match-page-wrapper commentary-page-wrapper
    # div - class:match-body
    # div - class:comment-container card
    # div - class:mb-5 comments-container-body
    # div - class: match-comment

    column_names = ['over', 'ball', 'bowler' , 'batsman' , 'runsOffBat' , 'isWicket', 'extras', 'commentary']

    match_data = pd.DataFrame(columns = column_names)

    level1 = soup.findAll('section' , attrs = {'id' : 'pane-main'})[0]
    level2 = level1.findAll('section', attrs = {'id' : 'main-container'})[0]
    level3 = level2.findAll('div', attrs = {'class' : 'match-page-wrapper commentary-page-wrapper'})[0]
    level4 = level3.findAll('div', attrs = {'class' : 'match-body'})[0]
    level5 = level4.findAll('div', attrs = {'class' : 'comment-container card'})[0]
    level6 = level5.findAll('div', attrs = {'class' : 'mb-5 comments-container-body'})[0]
    level7 = level6.findAll('div', attrs = {'class' : 'match-comment'})

    num = len(level7)

    for i in range(num):
        level8 = level7[i].findAll('div', attrs = {'class' : 'd-flex match-comment-padder align-items-center'})[0]

        level9 = level8.findAll('div', attrs = {'class' : 'col-2 col-md-1 col-lg-2 match-comment-run-col'})[0]
        overNum = level9.findAll('span', attrs = {'class' : 'match-comment-over'})[0].get_text()
        runsOffBat = level9.findAll('div', attrs = {'class' : 'match-comment-run-container'})[0].findAll('div', attrs = {'class' : 'match-comment-run'})[0].get_text()

        level9Text = level8.findAll('div', attrs = {'class' : 'col-14 col-md-15 col-lg-14'})[0]
        level10Text = level9Text.findAll('div', attrs = {'class' : 'match-comment-short-text'})[0]
        CommText = level10Text.get_text().split(',')[0]
        CommText_full = level10Text.get_text()
        bowler = CommText.split('to')[0]
        batsman = CommText.split('to')[1]
        over = overNum.split('.')[0]
        ball = overNum.split('.')[1]

        # Handle Wickets
        if (runsOffBat == 'W'):
            isWicket = 1
            runsOffBat = 0
        else:
            isWicket = 0

        # Handle Special cases 
        if (runsOffBat == '•'):
            runsOffBat = 0

        # Handle Extras 
        if (str(runsOffBat).endswith('w') or str(runsOffBat).endswith('nb')):
            extras = runsOffBat[:1]
            runsOffBat = 0 
        else: 
            extras = 0

        # Lot more possible cases to handle 

        match_data.loc[len(match_data.index)] = [over, ball, bowler, batsman, runsOffBat, isWicket, extras, CommText_full]
    
    return match_data

In [3]:
# get team 1 data
df1 = perform_scraping(1)

# get team 2 data
df2 = perform_scraping(2)

In [4]:
df1.head()

,over,ball,bowler,batsman,runsOffBat,isWicket,extras,commentary
0,19,6,Nalkande,Kumar,1,0,0,"Nalkande to Kumar, 1 run"
1,19,5,Nalkande,Manohar,0,1,0,"Nalkande to Manohar, OUT"
2,19,4,Nalkande,Suchith,0,1,0,"Nalkande to Suchith, OUT"
3,19,3,Nalkande,Sharath,0,1,0,"Nalkande to Sharath, OUT"
4,19,2,Nalkande,Joshi,0,1,0,"Nalkande to Joshi, OUT"


In [5]:
df1.tail()

,over,ball,bowler,batsman,runsOffBat,isWicket,extras,commentary
116,0,5,Nalkande,Kadam,0,0,0,"Nalkande to Kadam, no run"
117,0,4,Nalkande,Kadam,4,0,0,"Nalkande to Kadam, FOUR runs"
118,0,3,Nalkande,Kadam,0,0,0,"Nalkande to Kadam, no run"
119,0,2,Nalkande,Kadam,0,0,0,"Nalkande to Kadam, no run"
120,0,1,Nalkande,Kadam,0,0,0,"Nalkande to Kadam, no run"


In [6]:
df2.head()

,over,ball,bowler,batsman,runsOffBat,isWicket,extras,commentary
0,19,6,Patil,Nalkande,4,0,0,"Patil to Nalkande, FOUR runs"
1,19,5,Patil,Wankhade,1,0,0,"Patil to Wankhade, 1 run"
2,19,4,Patil,Nalkande,1b,0,0,"Patil to Nalkande, 1 bye"
3,19,3,Patil,Wankhade,1,0,0,"Patil to Wankhade, 1 run"
4,19,3,Patil,Wankhade,0,0,1,"Patil to Wankhade, 1 wide"


In [7]:
df2.tail()

,over,ball,bowler,batsman,runsOffBat,isWicket,extras,commentary
117,0,5,Darshan,Taide,1,0,0,"Darshan to Taide, 1 run"
118,0,4,Darshan,Taide,4,0,0,"Darshan to Taide, FOUR runs"
119,0,3,Darshan,Taide,6,0,0,"Darshan to Taide, SIX runs"
120,0,2,Darshan,Taide,4,0,0,"Darshan to Taide, FOUR runs"
121,0,1,Darshan,Taide,0,0,0,"Darshan to Taide, no run"
